In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
data = pd.read_csv('smoking_driking_dataset_Ver01.csv')

In [3]:
data.head()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,Male,35,170,75,90.0,1.0,1.0,1.0,1.0,120.0,...,126.0,92.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,Y
1,Male,30,180,80,89.0,0.9,1.2,1.0,1.0,130.0,...,148.0,121.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,N
2,Male,40,165,75,91.0,1.2,1.5,1.0,1.0,120.0,...,74.0,104.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,N
3,Male,50,175,80,91.0,1.5,1.2,1.0,1.0,145.0,...,104.0,106.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,N
4,Male,50,165,60,80.0,1.0,1.2,1.0,1.0,138.0,...,117.0,104.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,N


### Understanding the size of the dataset

In [4]:
data.shape

(991346, 24)

### Understanding the uniqueness of the data

In [5]:
data.nunique()

sex                    2
age                   14
height                13
weight                24
waistline            737
sight_left            24
sight_right           24
hear_left              2
hear_right             2
SBP                  171
DBP                  127
BLDS                 498
tot_chole            474
HDL_chole            223
LDL_chole            432
triglyceride        1657
hemoglobin           190
urine_protein          6
serum_creatinine     183
SGOT_AST             568
SGOT_ALT             594
gamma_GTP            940
SMK_stat_type_cd       3
DRK_YN                 2
dtype: int64

### Finding missing values 

In [6]:
data.isnull().sum()

sex                 0
age                 0
height              0
weight              0
waistline           0
sight_left          0
sight_right         0
hear_left           0
hear_right          0
SBP                 0
DBP                 0
BLDS                0
tot_chole           0
HDL_chole           0
LDL_chole           0
triglyceride        0
hemoglobin          0
urine_protein       0
serum_creatinine    0
SGOT_AST            0
SGOT_ALT            0
gamma_GTP           0
SMK_stat_type_cd    0
DRK_YN              0
dtype: int64

In [7]:
data = data.drop_duplicates(keep='first')
data[data.duplicated].shape

(0, 24)

### Description of dataset

In [8]:
data.describe()

,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,DBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
count,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,...,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000
mean,47.614529,162.240563,63.283884,81.233255,0.980833,0.978428,1.031495,1.030476,122.432360,76.052549,...,56.936984,113.037429,132.140030,14.229810,1.094221,0.860467,25.989424,25.755148,37.136152,1.608112
std,14.181346,9.282922,12.514101,11.850296,0.605954,0.604779,0.174652,0.171892,14.543083,9.889334,...,17.238578,35.842938,102.194762,1.584924,0.437719,0.480536,23.493668,26.308910,50.423811,0.818504
min,20.000000,130.000000,25.000000,8.000000,0.100000,0.100000,1.000000,1.000000,67.000000,32.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.100000,1.000000,1.000000,1.000000,1.000000
25%,35.000000,155.000000,55.000000,74.100000,0.700000,0.700000,1.000000,1.000000,112.000000,70.000000,...,46.000000,89.000000,73.000000,13.200000,1.000000,0.700000,19.000000,15.000000,16.000000,1.000000
50%,45.000000,160.000000,60.000000,81.000000,1.000000,1.000000,1.000000,1.000000,120.000000,76.000000,...,55.000000,111.000000,106.000000,14.300000,1.000000,0.800000,23.000000,20.000000,23.000000,1.000000
75%,60.000000,170.000000,70.000000,87.800000,1.200000,1.200000,1.000000,1.000000,131.000000,82.000000,...,66.000000,135.000000,159.000000,15.400000,1.000000,1.000000,28.000000,29.000000,39.000000,2.000000
max,85.000000,190.000000,140.000000,999.000000,9.900000,9.900000,2.000000,2.000000,273.000000,185.000000,...,8110.000000,5119.000000,9490.000000,25.000000,6.000000,98.000000,9999.000000,7210.000000,999.000000,3.000000


### Halving the dataset

In [9]:
sample_fraction = 0.001  
data = data.sample(frac=sample_fraction, random_state=42)

In [10]:
data.describe()

,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,DBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
count,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000,...,991.00000,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000,991.000000
mean,47.986882,162.225025,62.774975,80.659738,0.981130,0.949546,1.031282,1.036327,122.239152,75.887992,...,56.74672,113.292634,132.875883,14.214430,1.099899,0.853078,25.976791,25.340061,36.596367,1.591322
std,13.842871,9.046391,12.324273,9.312921,0.661371,0.449329,0.174165,0.187197,14.968935,10.320254,...,15.10841,34.863676,92.816827,1.660481,0.446207,0.204169,20.199472,20.061381,55.382148,0.811213
min,20.000000,135.000000,30.000000,55.000000,0.100000,0.100000,1.000000,1.000000,88.000000,48.000000,...,16.00000,29.000000,24.000000,7.400000,1.000000,0.400000,8.000000,5.000000,5.000000,1.000000
25%,40.000000,155.000000,55.000000,74.000000,0.700000,0.700000,1.000000,1.000000,111.000000,70.000000,...,46.00000,89.000000,74.000000,13.100000,1.000000,0.700000,19.000000,15.000000,16.000000,1.000000
50%,50.000000,165.000000,60.000000,81.000000,1.000000,1.000000,1.000000,1.000000,120.000000,76.000000,...,55.00000,112.000000,109.000000,14.300000,1.000000,0.800000,23.000000,20.000000,24.000000,1.000000
75%,60.000000,170.000000,70.000000,87.000000,1.200000,1.200000,1.000000,1.000000,130.000000,82.000000,...,65.00000,135.000000,165.000000,15.400000,1.000000,1.000000,28.000000,29.000000,40.000000,2.000000
max,85.000000,185.000000,110.000000,115.000000,9.900000,9.900000,2.000000,2.000000,190.000000,150.000000,...,135.00000,266.000000,955.000000,18.300000,5.000000,2.700000,474.000000,281.000000,999.000000,3.000000


# Preprocessing of the dataset

### Converting categorical values

In [11]:
data['sex'] = data['sex'].map({'Male': 1, 'Female': 0})
data['DRK_YN'] = data['DRK_YN'].map({'Y': 1, 'N': 0})


X = data.drop(['DRK_YN', 'SMK_stat_type_cd'], axis=1)
y = data['DRK_YN']

display(data.head())

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
857155,1,45,165,70,91.0,0.9,1.5,1.0,1.0,138.0,...,69.0,142.0,15.8,1.0,0.8,28.0,20.0,60.0,3.0,1
248823,1,75,165,60,85.0,0.6,0.7,1.0,1.0,165.0,...,43.0,205.0,14.9,1.0,0.8,13.0,16.0,23.0,1.0,0
903710,1,35,165,65,76.0,1.2,1.2,1.0,1.0,134.0,...,138.0,107.0,16.6,1.0,0.9,17.0,19.0,33.0,3.0,1
59866,0,60,145,50,80.0,0.6,0.9,1.0,1.0,116.0,...,69.0,251.0,12.2,5.0,1.3,21.0,12.0,13.0,1.0,0
192679,0,55,155,45,60.0,1.5,1.0,1.0,1.0,100.0,...,110.0,101.0,13.2,1.0,0.9,16.0,12.0,14.0,1.0,0


In [12]:
# Verification
# Check if the specified columns are dropped from X
print("Columns in X after dropping: ", X.columns.tolist())
print("First few rows of X:\n", X.head())

# Check if y contains the correct target variable
print("First few rows of y:\n", y.head())

print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)

Columns in X after dropping:  ['sex', 'age', 'height', 'weight', 'waistline', 'sight_left', 'sight_right', 'hear_left', 'hear_right', 'SBP', 'DBP', 'BLDS', 'tot_chole', 'HDL_chole', 'LDL_chole', 'triglyceride', 'hemoglobin', 'urine_protein', 'serum_creatinine', 'SGOT_AST', 'SGOT_ALT', 'gamma_GTP']
First few rows of X:
         sex  age  height  weight  waistline  sight_left  sight_right  \
857155    1   45     165      70       91.0         0.9          1.5   
248823    1   75     165      60       85.0         0.6          0.7   
903710    1   35     165      65       76.0         1.2          1.2   
59866     0   60     145      50       80.0         0.6          0.9   
192679    0   55     155      45       60.0         1.5          1.0   

        hear_left  hear_right    SBP  ...  tot_chole  HDL_chole  LDL_chole  \
857155        1.0         1.0  138.0  ...      149.0       52.0       69.0   
248823        1.0         1.0  165.0  ...      118.0       34.0       43.0   
903710      

In [13]:
for feature in data:
    print(feature)

sex
age
height
weight
waistline
sight_left
sight_right
hear_left
hear_right
SBP
DBP
BLDS
tot_chole
HDL_chole
LDL_chole
triglyceride
hemoglobin
urine_protein
serum_creatinine
SGOT_AST
SGOT_ALT
gamma_GTP
SMK_stat_type_cd
DRK_YN


In [14]:
# Checking if there are outliers
for feature in data:
    
    Q1 = data[feature].quantile(0.25)
    Q3 = data[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    
    if data[(data[feature] > upper)].any(axis=None):
        print(feature,"yes")
    else:
        print(feature, "no")

sex no
age no
height no
weight yes
waistline yes
sight_left yes
sight_right yes
hear_left yes
hear_right yes
SBP yes
DBP yes
BLDS yes
tot_chole yes
HDL_chole yes
LDL_chole yes
triglyceride yes
hemoglobin no
urine_protein yes
serum_creatinine yes
SGOT_AST yes
SGOT_ALT yes
gamma_GTP yes
SMK_stat_type_cd no
DRK_YN no


In [15]:
outliers_list = []

for feature in data.columns:
    Q1 = data[feature].quantile(0.05)
    Q3 = data[feature].quantile(0.95)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = data[(data[feature] < lower) | (data[feature] > upper)]
    
    if not outliers.empty:
        outliers = outliers.copy() 
        outliers['outlier_feature'] = feature 
        outliers_list.append(outliers)
        print(f"{feature}: yes")
    else:
        print(f"{feature}: no")

sex: no
age: no
height: no
weight: no
waistline: no
sight_left: yes
sight_right: yes
hear_left: yes
hear_right: yes
SBP: no
DBP: yes
BLDS: yes
tot_chole: no
HDL_chole: no
LDL_chole: no
triglyceride: yes
hemoglobin: no
urine_protein: yes
serum_creatinine: yes
SGOT_AST: yes
SGOT_ALT: yes
gamma_GTP: yes
SMK_stat_type_cd: no
DRK_YN: no


In [16]:
if outliers_list:
    all_outliers = pd.concat(outliers_list)
    all_outliers.drop_duplicates()
    outlier_count = all_outliers.shape[0]
    
    print(f"\nNumber of extreme outliers in the dataset: {outlier_count}")
    print("\nSome rows with extreme outliers:")
    print(all_outliers.head())
    
else:
    print("\nNo outliers found in the dataset.")


Number of extreme outliers in the dataset: 113

Some rows with extreme outliers:
        sex  age  height  weight  waistline  sight_left  sight_right  \
928218    0   85     145      50       82.8         9.9          0.7   
292385    1   70     165      65       81.0         9.9          1.0   
201803    1   45     165      60       80.0         9.9          1.2   
348052    0   60     155      60       78.0         9.9          0.2   
815459    0   80     150      50       79.0         0.5          9.9   

        hear_left  hear_right    SBP  ...  triglyceride  hemoglobin  \
928218        1.0         1.0  148.0  ...         132.0        14.1   
292385        1.0         1.0  130.0  ...          65.0        13.3   
201803        1.0         1.0  129.0  ...         150.0        14.0   
348052        1.0         1.0  120.0  ...          48.0        14.0   
815459        1.0         1.0  152.0  ...         135.0        13.0   

        urine_protein  serum_creatinine  SGOT_AST  SGOT_AL

### Removing extreme outliers

In [17]:
data = data.drop(all_outliers.index)
data = data.drop_duplicates()
print("\nDataFrame after removing extreme outliers:")
print(data.head())


DataFrame after removing extreme outliers:
        sex  age  height  weight  waistline  sight_left  sight_right  \
857155    1   45     165      70       91.0         0.9          1.5   
248823    1   75     165      60       85.0         0.6          0.7   
903710    1   35     165      65       76.0         1.2          1.2   
192679    0   55     155      45       60.0         1.5          1.0   
653978    1   35     175      75       83.0         1.2          1.5   

        hear_left  hear_right    SBP  ...  LDL_chole  triglyceride  \
857155        1.0         1.0  138.0  ...       69.0         142.0   
248823        1.0         1.0  165.0  ...       43.0         205.0   
903710        1.0         1.0  134.0  ...      138.0         107.0   
192679        1.0         1.0  100.0  ...      110.0         101.0   
653978        1.0         1.0  110.0  ...      142.0          62.0   

        hemoglobin  urine_protein  serum_creatinine  SGOT_AST  SGOT_ALT  \
857155        15.8         

In [18]:
data.describe()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
count,904.000000,904.000000,904.000000,904.000000,904.000000,904.000000,904.000000,904.0,904.0,904.000000,...,904.000000,904.000000,904.000000,904.000000,904.000000,904.000000,904.000000,904.000000,904.000000,904.000000
mean,0.519912,46.897124,162.582965,62.887168,80.449004,0.959403,0.953982,1.0,1.0,121.453540,...,113.339602,128.319690,14.219469,1.065265,0.849779,24.360619,24.097345,33.216814,1.598451,0.485619
std,0.499880,13.190882,8.959969,12.189192,9.319483,0.335893,0.346835,0.0,0.0,14.750192,...,34.208998,82.643523,1.649510,0.296060,0.194612,8.915404,15.487194,29.468783,0.817336,0.500070
min,0.000000,20.000000,135.000000,35.000000,55.000000,0.100000,0.100000,1.0,1.0,88.000000,...,29.000000,24.000000,7.400000,1.000000,0.400000,8.000000,5.000000,5.000000,1.000000,0.000000
25%,0.000000,40.000000,155.000000,55.000000,74.000000,0.800000,0.700000,1.0,1.0,110.000000,...,89.000000,73.000000,13.100000,1.000000,0.700000,19.000000,15.000000,16.000000,1.000000,0.000000
50%,1.000000,45.000000,165.000000,60.000000,81.000000,1.000000,1.000000,1.0,1.0,120.000000,...,112.000000,105.500000,14.300000,1.000000,0.800000,22.000000,20.000000,23.000000,1.000000,0.000000
75%,1.000000,55.000000,170.000000,70.000000,86.200000,1.200000,1.200000,1.0,1.0,130.000000,...,135.000000,163.000000,15.400000,1.000000,1.000000,28.000000,29.000000,39.000000,2.000000,1.000000
max,1.000000,85.000000,185.000000,110.000000,115.000000,2.000000,2.000000,1.0,1.0,190.000000,...,266.000000,651.000000,18.200000,3.000000,1.500000,79.000000,123.000000,226.000000,3.000000,1.000000


In [19]:
data.to_csv('preprocessed_dataset.csv', index=False)

# Feature engineering

In [20]:
X = data.drop(['DRK_YN', 'SMK_stat_type_cd'], axis=1)
y = data['DRK_YN'] 



# Split the data into training and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the training + validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

# Transform the validation and test data using the fitted scaler
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

#model_rf_feng = RandomForestClassifier()
#model_rf_feng.fit(X_train, y_train)

#joblib.dump(model_rf_feng, 'random_forest_model_feng.pkl')
model_rf_feng = joblib.load('random_forest_model_feng.pkl')

y_pred = model_rf_feng.predict(X_test)

# Classification Report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95        97
           1       0.94      0.94      0.94        84

    accuracy                           0.94       181
   macro avg       0.94      0.94      0.94       181
weighted avg       0.94      0.94      0.94       181



### 60 % for train, 20% for validation, 20% for test

In [21]:
print(len(X_train), len(X_test), len(X_val))

542 181 181


In [22]:
num_target_value_1 = (y_train == 1).sum()
num_target_value_0 = (y_train == 0).sum()

# Print the result
print(f"Number of samples in y_train with the target value 1: {num_target_value_1}")
print(f"Number of samples in y_train with the target value 0: {num_target_value_0}")

Number of samples in y_train with the target value 1: 259
Number of samples in y_train with the target value 0: 283


### Train dataset does not need oversampling

In [23]:
!pip install mlxtend

### Forward Feature Selection

In [24]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error

rf = RandomForestClassifier()

ffs = SequentialFeatureSelector(rf, k_features='best', forward=True, n_jobs=-1)
ffs.fit(X_train_scaled, y_train)

features = list(ffs.k_feature_names_)

rf.fit(X_train_scaled[features], y_train)

y_pred = rf.predict(X_train_scaled[features])

analysis_df = X_train_scaled[features].copy()
analysis_df['Actual'] = y_train
analysis_df['Predicted'] = y_pred

print(analysis_df.head())

# Mean Squared Error for evaluation
mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error: {mse}')

analysis_df.to_csv('feature_selection_analysis.csv', index=False)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### Backward Feature Elimination

In [ ]:
rf = RandomForestClassifier()

bfs = SequentialFeatureSelector(rf, k_features='best', forward=False, n_jobs=-1)
bfs.fit(X_train_scaled, y_train)

selected_features = list(bfs.k_feature_names_)
removed_features = list(set(X_train.columns) - set(selected_features))

rf.fit(X_train_scaled[selected_features], y_train)

y_pred = rf.predict(X_train_scaled[selected_features])

# Combine actual and predicted values into a DataFrame
analysis_df = X_train_scaled[selected_features].copy()
analysis_df['Actual'] = y_train
analysis_df['Predicted'] = y_pred

print(analysis_df.head())

# Mean Squared Error
mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error: {mse}')

analysis_df.to_csv('backward_feature_selection_analysis.csv', index=False)

# Removed features
print(f'Removed Features: {removed_features}')

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

# Correlation Analysis
corr_matrix = data.corr()
print(corr_matrix['DRK_YN'][['LDL_chole', 'HDL_chole']])

# Coefficient Analysis
lr.fit(X_train, y_train)
coefficients = pd.Series(lr.coef_, index=X_train.columns)
print(coefficients[['LDL_chole', 'HDL_chole']])

# Cross-Validation Scores
features_with_hdl_ldl = selected_features + ['LDL_chole', 'HDL_chole']
cross_val_scores = cross_val_score(lr, X_train[features_with_hdl_ldl], y_train, cv=5, scoring='neg_mean_squared_error')
print("Cross-Validation MSE with LDL and HDL:", -np.mean(cross_val_scores))

cross_val_scores_without_hdl_ldl = cross_val_score(lr, X_train[selected_features], y_train, cv=5, scoring='neg_mean_squared_error')
print("Cross-Validation MSE without LDL and HDL:", -np.mean(cross_val_scores_without_hdl_ldl))

# Visualize correlations
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Features')
plt.show()


### Exhaustive Feature Selection

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector

from sklearn.ensemble import RandomForestClassifier

efs = ExhaustiveFeatureSelector(RandomForestClassifier(),
                               min_features=5,
                               max_features=5,
                               scoring='roc_auc',
                               cv=2)
efs=efs.fit(X,y)

selected_features = X_train.columns[list(efs.best_idx_)]
print(selected_features)

print(efs.best_score_)